# 12) 감성분석(KO BERT)

In [1]:
# 코랩에서 실행

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.8 MB/s eta 0:00:00


In [3]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.3 MB/s eta 0:00:00


In [4]:
import tensorflow as tf

import numpy as np

import pandas as pd

import json

import numpy as np

import pandas as pd

from tqdm import tqdm

import os

import sentencepiece as spm

In [5]:
import os

from google.colab import drive

# drive.mount('/content/gdrive/')

In [6]:
# 네이버 영화 감성분석 데이터 다운로드

!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 19.42 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Updating files: 100% (14737/14737), done.


In [7]:
os.listdir('nsmc')

['.git',
 'raw',
 'code',
 'ratings_test.txt',
 'README.md',
 'synopses.json',
 'ratings_train.txt',
 'ratings.txt']

In [8]:
train = pd.read_table("nsmc/"+"ratings_train.txt")

test = pd.read_table("nsmc/"+"ratings_test.txt")

In [9]:
train=train[:10000]

test=test[:10000]

In [10]:
train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [11]:
# KOBERT : SKT에서 개발한 한국어 모형

import logging

import os

import unicodedata

from shutil import copyfile

from transformers import PreTrainedTokenizer

logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",

                     "vocab_txt": "vocab.txt"}

PRETRAINED_VOCAB_FILES_MAP = {

    "vocab_file": {

        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",

        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",

        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"

    },

    "vocab_txt": {

        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",

        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",

        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"

    }

}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {

    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,

    "monologg/distilkobert": 512

}

PRETRAINED_INIT_CONFIGURATION = {

    "monologg/kobert": {"do_lower_case": False},

    "monologg/kobert-lm": {"do_lower_case": False},

    "monologg/distilkobert": {"do_lower_case": False}

}

SPIECE_UNDERLINE = u'▁'

class KoBertTokenizer(PreTrainedTokenizer):

    vocab_files_names = VOCAB_FILES_NAMES

    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP

    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION

    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(

            self,

            vocab_file,

            vocab_txt,

            do_lower_case=False,

            remove_space=True,

            keep_accents=False,

            unk_token="[UNK]",

            sep_token="[SEP]",

            pad_token="[PAD]",

            cls_token="[CLS]",

            mask_token="[MASK]",

            **kwargs):

        super().__init__(

            unk_token=unk_token,

            sep_token=sep_token,

            pad_token=pad_token,

            cls_token=cls_token,

            mask_token=mask_token,
            **kwargs

        )

        # Build vocab

        self.token2idx = dict()

        self.idx2token = []

        with open(vocab_txt, 'r', encoding='utf-8') as f:

            for idx, token in enumerate(f):

                token = token.strip()

                self.token2idx[token] = idx

                self.idx2token.append(token)

        try:

            import sentencepiece as spm

        except ImportError:

            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"

                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case

        self.remove_space = remove_space

        self.keep_accents = keep_accents

        self.vocab_file = vocab_file

        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()

        self.sp_model.Load(vocab_file)

    @property

    def vocab_size(self):

        return len(self.idx2token)

    def get_vocab(self):

        return dict(self.token2idx, **self.added_tokens_encoder)

    def __getstate__(self):

        state = self.__dict__.copy()

        state["sp_model"] = None

        return state
    def __setstate__(self, d):

        self.__dict__ = d

        try:

            import sentencepiece as spm

        except ImportError:

            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"

                           "pip install sentencepiece")

        self.sp_model = spm.SentencePieceProcessor()

        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):

        if self.remove_space:

            outputs = " ".join(inputs.strip().split())

        else:

            outputs = inputs

        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:

            outputs = unicodedata.normalize('NFKD', outputs)

            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])

        if self.do_lower_case:

            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):

        text = self.preprocess_text(text)

        if not sample:

            pieces = self.sp_model.EncodeAsPieces(text)

        else:

            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)

        new_pieces = []

        for piece in pieces:

            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():

                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:

                    if len(cur_pieces[0]) == 1:

                        cur_pieces = cur_pieces[1:]

                    else:

                        cur_pieces[0] = cur_pieces[0][1:]

                cur_pieces.append(piece[-1])

                new_pieces.extend(cur_pieces)

            else:

                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):

        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):

        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):

        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()

        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):

        if token_ids_1 is None:

            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]

        cls = [self.cls_token_id]

        sep = [self.sep_token_id]

        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):

        if already_has_special_tokens:

            if token_ids_1 is not None:

                raise ValueError(

                    "You should not supply a second sequence if the provided sequence of "

                    "ids is already formated with special tokens for the model."

                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:

            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]

        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):

        sep = [self.sep_token_id]

        cls = [self.cls_token_id]

        if token_ids_1 is None:

            return len(cls + token_ids_0 + sep) * [0]

        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):

        if not os.path.isdir(save_directory):

            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))

            return

        # 1. Save sentencepiece model

        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):

            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt

        index = 0

        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])

        with open(out_vocab_txt, "w", encoding="utf-8") as writer:

            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):

                if index != token_index:

                    logger.warning(

                        "Saving vocabulary to {}: vocabulary indices are not consecutive."

                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)

                    )

                    index = token_index

                writer.write(token + "\n")

                index += 1
        return out_vocab_model, out_vocab_txt

In [12]:

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [13]:
def convert_data(data_df):

    global tokenizer

    SEQ_LEN = 64

    tokens, masks, segments, targets = [], [], [], []

    for i in tqdm(range(len(data_df))):

        token = tokenizer.encode(data_df[DATA_COLUMN][i], truncation=True, padding='max_length', max_length=SEQ_LEN)

        # mask : 패딩이 아닌 부분은 1, 패딩인 부분은 0으로

        num_zeros = token.count(0)

        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros

        # segment : 문장의 전후관계를 구분, 연결되는 문장이면 1, 그렇지 않으면 0, 여기서는 문장이 1개이므로 모두 0

        segment = [0]*SEQ_LEN

        tokens.append(token)

        masks.append(mask)

        segments.append(segment)

        targets.append(data_df[LABEL_COLUMN][i])

    tokens = np.array(tokens)

    masks = np.array(masks)

    segments = np.array(segments)

    targets = np.array(targets)

    return [tokens, masks, segments], targets

def load_data(pandas_dataframe):

        return out_vocab_model, out_vocab_txt

##################################################

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

##################################################

def convert_data(data_df):

    global tokenizer

    SEQ_LEN = 64

    tokens, masks, segments, targets = [], [], [], []

    for i in tqdm(range(len(data_df))):

        token = tokenizer.encode(data_df[DATA_COLUMN][i], truncation=True, padding='max_length', max_length=SEQ_LEN)

        # mask : 패딩이 아닌 부분은 1, 패딩인 부분은 0으로

        num_zeros = token.count(0)

        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros

        # segment : 문장의 전후관계를 구분, 연결되는 문장이면 1, 그렇지 않으면 0, 여기서는 문장이 1개이므로 모두 0

        segment = [0]*SEQ_LEN

        tokens.append(token)

        masks.append(mask)

        segments.append(segment)

        targets.append(data_df[LABEL_COLUMN][i])

    tokens = np.array(tokens)

    masks = np.array(masks)

    segments = np.array(segments)

    targets = np.array(targets)

    return [tokens, masks, segments], targets

def load_data(pandas_dataframe):

    data_df = pandas_dataframe

    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)

    data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)

    data_x, data_y = convert_data(data_df)

    return data_x, data_y

SEQ_LEN = 64

BATCH_SIZE = 32

DATA_COLUMN = "document"

LABEL_COLUMN = "label"

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [14]:
# BERT input 형태로 변환

train_x, train_y = load_data(train)

test_x, test_y = load_data(test)

100%|██████████| 10000/10000 [00:03<00:00, 2657.42it/s]


In [15]:
import transformers

model = transformers.TFBertModel.from_pretrained("monologg/kobert", from_pt=True)

model.summary()

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  92186880  
                                                                 
Total params: 92,186,880
Trainable params: 92,186,880
Non-trainable params: 0
_________________________________________________________________


In [16]:
token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')

mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')

segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')

bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

In [17]:
bert_outputs[0]

<KerasTensor: shape=(None, 64, 768) dtype=float32 (created by layer 'tf_bert_model')>

In [18]:
bert_outputs[1]

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model')>

In [19]:
bert_outputs = bert_outputs[1]

bert_outputs

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model')>

In [20]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 7.2 MB/s eta 0:00:00


In [21]:
import tensorflow_addons as tfa

# total_steps : batch size * epochs

opt = tfa.optimizers.RectifiedAdam(lr=5.0e-5, total_steps = 2344*2, warmup_proportion=0.1, min_lr=1e-5, epsilon=1e-08, clipnorm=1.0)

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/optimizers/rectified_adam.py:121: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [22]:
sentiment_drop = tf.keras.layers.Dropout(0.5)(bert_outputs)

sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(sentiment_drop)

sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)

sentiment_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

In [23]:
sentiment_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 64)]         0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 64)]         0           []                               
                                                                                                  
 input_segment (InputLayer)     [(None, 64)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  92186880    ['input_word_ids[0][0]',         
                                thPoolingAndCrossAt               'input_masks[0][0]',        

In [24]:
sentiment_model.fit(train_x, train_y, epochs=2, shuffle=True, batch_size=64,

                    validation_data=(test_x, test_y))

Epoch 1/2
157/157 [==============================] - 221s 1s/step - loss: 0.6994 - accuracy: 0.5095 - val_loss: 0.6851 - val_accuracy: 0.5399
Epoch 2/2
157/157 [==============================] - 169s 1s/step - loss: 0.6189 - accuracy: 0.6605 - val_loss: 0.5006 - val_accuracy: 0.7722


In [25]:
def predict_convert_data(data_df):

    global tokenizer

    tokens, masks, segments = [], [], []

    for i in tqdm(range(len(data_df))):

        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, truncation=True, padding='max_length')

        num_zeros = token.count(0)

        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros

        segment = [0]*SEQ_LEN

        tokens.append(token)

        segments.append(segment)
        masks.append(mask)

    tokens = np.array(tokens)

    masks = np.array(masks)

    segments = np.array(segments)

    return [tokens, masks, segments]

def predict_load_data(pandas_dataframe):

    data_df = pandas_dataframe

    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)

    data_x = predict_convert_data(data_df)

    return data_x

In [26]:
test_set = predict_load_data(test)

100%|██████████| 10000/10000 [00:02<00:00, 4423.70it/s]


In [27]:
test_set

[array([[   2,  517, 5515, ...,    1,    1,    1],
        [   2,  650,  278, ...,    1,    1,    1],
        [   2, 2145, 6844, ...,    1,    1,    1],
        ...,
        [   2, 1055,  517, ...,    1,    1,    1],
        [   2,  517, 5993, ...,    1,    1,    1],
        [   2,  835, 5725, ...,    1,    1,    1]]),
 array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [28]:
preds = sentiment_model.predict(test_set)

preds

313/313 [==============================] - 53s 148ms/step


array([[0.68988264],
       [0.35565993],
       [0.2139305 ],
       ...,
       [0.20819645],
       [0.21082607],
       [0.2170701 ]], dtype=float32)

In [29]:
from sklearn.metrics import classification_report

y_true = test['label']

print(classification_report(y_true, np.round(preds,0)))



              precision    recall  f1-score   support

           0       0.72      0.89      0.79      4939
           1       0.86      0.66      0.74      5061

    accuracy                           0.77     10000
   macro avg       0.79      0.77      0.77     10000
weighted avg       0.79      0.77      0.77     10000



In [30]:
def sentence_convert_data(data):

    global tokenizer

    tokens, masks, segments = [], [], []

    token = tokenizer.encode(data, max_length=SEQ_LEN, truncation=True, padding='max_length')

    num_zeros = token.count(0)

    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros

    segment = [0]*SEQ_LEN

    tokens.append(token)
    segments.append(segment)

    masks.append(mask)

    tokens = np.array(tokens)

    masks = np.array(masks)

    segments = np.array(segments)

    return [tokens, masks, segments]

def movie_evaluation_predict(sentence):

    data_x = sentence_convert_data(sentence)

    predict = sentiment_model.predict(data_x)

    predict_value = np.ravel(predict)

    predict_answer = np.round(predict_value,0).item()

    if predict_answer == 0:

      print("negative %.2f%%" % ((1-predict_value)*100))

    elif predict_answer == 1:

      print("positive %.2f%%" % (predict_value*100))

In [31]:
movie_evaluation_predict("연기는 잔잔하게 볼 만 합니다 ")

1/1 [==============================] - 0s 49ms/step
positive 81.46%


In [32]:
movie_evaluation_predict("영화의 주제가 뭔지 모르겠음")

1/1 [==============================] - 0s 59ms/step
negative 78.26%


In [33]:
movie_evaluation_predict("익살스런 연기가 돋보였던 영화")

1/1 [==============================] - 0s 49ms/step
positive 79.47%


In [34]:
movie_evaluation_predict("기대보다는 스토리가 큰 감흥은 없습니다.")

1/1 [==============================] - 0s 50ms/step
negative 73.29%
